In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

gpu_num = 0  # Use "" to use the CPU, Use 0 to select first GPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel('ERROR')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from dmimo.baseline import sim_baseline, sim_baseline_all

#### dMIMO simulations for baseline scenario using SVD and ZF precoding

In [ ]:
modulation_orders = [2, 4, 6]  # modulation order: 2/4/6 for QPSK/16QAM/64QAM
csi_delay = 1  # feedback delay in number of subframe
num_modulations = len(modulation_orders)

ber = np.zeros((2, num_modulations))
ldpc_ber = np.zeros((2, num_modulations))
goodput= np.zeros((2, num_modulations))
throughput = np.zeros((2, num_modulations))
for k in range(num_modulations):
    rst_svd = sim_baseline_all(precoding_method="SVD", csi_delay=csi_delay, num_bits_per_symbol=modulation_orders[k])
    ber[0, k] = rst_svd[0]
    ldpc_ber[0, k] = rst_svd[1]
    goodput[0, k] = rst_svd[2]
    throughput[0, k] = rst_svd[3]
    rst_zf = sim_baseline_all(precoding_method="ZF", csi_delay=csi_delay, num_bits_per_symbol=modulation_orders[k])
    ber[1, k] = rst_zf[0]
    ldpc_ber[1, k] = rst_zf[1]
    goodput[1, k] = rst_zf[2]
    throughput[0, k] = rst_zf[3]

#### Compare performance results

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,4))

ax[0].set_title("SU-MIMO (Baseline)")
ax[0].set_xlabel('Modulation (bits/symbol)')
ax[0].set_ylabel('BER')
im = ax[0].plot(modulation_orders, ber.transpose(), 'o-')
im = ax[0].legend(['SVD', 'ZF'])

ax[1].set_title("SU-MIMO (Baseline)")
ax[1].set_xlabel('Modulation (bits/symbol)')
ax[1].set_ylabel('Coded BER')
im = ax[1].plot(modulation_orders, ldpc_ber.transpose(), 'd-')
im = ax[1].legend(['SVD', 'ZF'])

ax[2].set_title("SU-MIMO (Baseline)")
ax[2].set_xlabel('Modulation (bits/symbol)')
ax[2].set_ylabel('Goodput (Mbps)')
im = ax[2].plot(modulation_orders, goodput.transpose(), 's-')
im = ax[2].legend(['SVD', 'ZF'])

In [ ]:
ber_svd, bits_svd, xh_svd = sim_baseline(precoding_method="SVD", csi_delay=1, num_bits_per_symbol=4)

In [ ]:
print("Uncoded BER: {}, LDPC BER: {}".format(ber_svd[0], ber_svd[1]))

In [ ]:
x1a = xh_svd[1,0,0]
x1b = xh_svd[1,0,2]
x1c = xh_svd[1,0,3]

fig, ax = plt.subplots(1,3, figsize=(15,4))
ax[0].set_xlim([-2, 2])
ax[0].set_ylim([-2, 2])
im = ax[0].scatter(x1a.real, x1a.imag)
ax[1].set_xlim([-2, 2])
ax[1].set_ylim([-2, 2])
im = ax[1].scatter(x1b.real, x1b.imag)
ax[2].set_xlim([-2, 2])
ax[2].set_ylim([-2, 2])
im = ax[2].scatter(x1c.real, x1c.imag)